In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import os
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_pacf, plot_acf
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.statespace.sarimax import SARIMAX

# Importing data

In [20]:
os.getcwd()

'/Users/michael.kuntz/code/HaukeFock/predict_energy_generation/notebooks'

In [21]:
local_path = '/Users/michael.kuntz/code/HaukeFock/predict_energy_generation/raw_data/Target/energy_generation_data'

In [22]:
for path, dirs, files in os.walk(local_path):
    print(path)
    for f in files:
        print(f)

/Users/michael.kuntz/code/HaukeFock/predict_energy_generation/raw_data/Target/energy_generation_data
.DS_Store
df_amprion.xlsx
df_deutschland.csv
df_tennet.xlsx
df_transnet_bw.xlsx
df_50hertz.xlsx


In [29]:
daily_data_path = '/Users/michael.kuntz/code/HaukeFock/predict_energy_generation/raw_data/Target/Daily/energy_generation_data/df_deutschland.csv'
hourly_data_path = '/Users/michael.kuntz/code/HaukeFock/predict_energy_generation/raw_data/Target/Hourly/energy_generation_data/df_deutschland.csv'

In [30]:
daily_df = pd.read_csv(daily_data_path)

daily_df['Date'] = pd.to_datetime(daily_df['Date'])
daily_df.set_index('Date', inplace=True)

daily_df.head()

,Wind offshore[MWh],Wind onshore[MWh],Photovoltaics[MWh],region
Date,,,,
2015-01-01,11.989583,1278.427083,75.531250,50hertz
2015-01-02,9.947917,2555.510417,17.677083,50hertz
2015-01-03,11.281250,2308.958333,27.354167,50hertz
2015-01-04,11.354167,2079.041667,49.750000,50hertz
2015-01-05,4.104167,1333.343750,20.187500,50hertz


In [31]:
hourly_df = pd.read_csv(hourly_data_path)

hourly_df['Date'] = pd.to_datetime(hourly_df['Date'])
hourly_df.set_index('Date', inplace=True)

hourly_df.head()

,Wind offshore[MWh],Wind onshore[MWh],Photovoltaics[MWh],region
Date,,,,
2015-01-01 00:00:00,12.0,910.25,0.0,50hertz
2015-01-01 01:00:00,12.0,1004.00,0.0,50hertz
2015-01-01 02:00:00,12.0,1058.00,0.0,50hertz
2015-01-01 03:00:00,12.0,998.50,0.0,50hertz
2015-01-01 04:00:00,12.0,952.25,0.0,50hertz


# Exploring data

In [35]:
hourly_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 210528 entries, 2015-01-01 00:00:00 to 2021-01-01 23:00:00
Data columns (total 4 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Wind offshore[MWh]  105250 non-null  float64
 1   Wind onshore[MWh]   210504 non-null  float64
 2   Photovoltaics[MWh]  210504 non-null  float64
 3   region              210528 non-null  object 
dtypes: float64(3), object(1)
memory usage: 8.0+ MB


In [36]:
hourly_df.describe()

,Wind offshore[MWh],Wind onshore[MWh],Photovoltaics[MWh]
count,105250.000000,210504.000000,210504.000000
mean,255.739760,607.539364,278.317210
std,322.525403,719.092184,466.549905
min,0.000000,0.000000,0.000000
25%,27.750000,82.750000,0.000000
50%,98.000000,313.750000,6.750000
75%,369.250000,891.312500,394.000000
max,1520.000000,3968.750000,3075.250000


In [39]:
hourly_df.isna().sum()

Wind offshore[MWh]    105278
Wind onshore[MWh]         24
Photovoltaics[MWh]        24
region                     0
dtype: int64